In [2]:
train_input="s3://sagemaker-us-east-2-363557075783/insurance/insurance.csv"

In [3]:
import sagemaker
import boto3
from sagemaker import Session

# Let's create a Sagemaker session
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
print(role)

arn:aws:iam::363557075783:role/service-role/AmazonSageMaker-ExecutionRole-20210929T104599


In [4]:
bucket = sagemaker_session.default_bucket()
prefix = "insurance"

In [5]:
bucket

'sagemaker-us-east-2-363557075783'

In [6]:
from sagemaker.sklearn.estimator import SKLearn

script_path = 'preprocessing.py'
FRAMEWORK_VERSION = "0.23-1"
sklearn_preprocessor = SKLearn(
    entry_point=script_path,
    role=role,
    framework_version=FRAMEWORK_VERSION,
    train_instance_type="ml.m4.xlarge",
    sagemaker_session=sagemaker_session)

sklearn_preprocessor.fit({'train': train_input})

train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


2021-10-27 03:21:24 Starting - Starting the training job...
2021-10-27 03:21:26 Starting - Launching requested ML instancesProfilerReport-1635304884: InProgress
......
2021-10-27 03:22:54 Starting - Preparing the instances for training.........
2021-10-27 03:24:22 Downloading - Downloading input data...
2021-10-27 03:24:55 Training - Downloading the training image...
2021-10-27 03:25:17 Training - Training image download completed. Training in progress..2021-10-27 03:25:18,771 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2021-10-27 03:25:18,775 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-10-27 03:25:18,788 sagemaker_sklearn_container.training INFO     Invoking user training script.
2021-10-27 03:25:19,088 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-10-27 03:25:25,338 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-10-27

In [7]:
# Define a SKLearn Transformer from the trained SKLearn Estimator
transformer = sklearn_preprocessor.transformer(
    instance_count=1, 
    instance_type='ml.m4.xlarge',
    assemble_with = 'Line',
    accept = 'text/csv')
# Preprocess training input
transformer.transform(train_input, content_type='text/csv')
print('Waiting for transform job: ' + transformer.latest_transform_job.job_name)
transformer.wait()
preprocessed_train = transformer.output_path

.................................
2021-10-27 03:31:24,224 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
2021-10-27 03:31:24,227 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
2021-10-27 03:31:24,228 INFO - sagemaker-containers - nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;

worker_rlimit_nofile 4096;

events {
  worker_connections 2048;
}

2021-10-27 03:31:24,224 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
2021-10-27 03:31:24,227 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
2021-10-27 03:31:24,228 INFO - sagemaker-containers - nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;

worker_rlimit_nofile 4096;

events {
  worker_connections 2048;
}

http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;

  upstream gu

In [8]:
import boto3
from sagemaker.image_uris import retrieve

ll_image = retrieve("linear-learner", boto3.Session().region_name)

In [9]:
s3_ll_output_key_prefix = "training_output"
s3_ll_output_location = 's3://{}/{}/{}/{}'.format(bucket, prefix, s3_ll_output_key_prefix, 'll_model')

ll_estimator = sagemaker.estimator.Estimator(
    ll_image,
    role, 
    train_instance_count=1, 
    train_instance_type='ml.m4.xlarge',
    train_volume_size = 20,
    train_max_run = 3600,
    input_mode= 'File',
    output_path=s3_ll_output_location,
    sagemaker_session=sagemaker_session)

ll_estimator.set_hyperparameters(feature_dim=11, predictor_type='regressor', mini_batch_size=32)

ll_train_data = sagemaker.session.s3_input(
    preprocessed_train, 
    distribution='FullyReplicated',
    content_type='text/csv', 
    s3_data_type='S3Prefix')

data_channels = {'train': ll_train_data}
ll_estimator.fit(inputs=data_channels)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_run has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_volume_size has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The class sagemaker.session.s3_input has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


2021-10-27 03:32:05 Starting - Starting the training job...
2021-10-27 03:32:28 Starting - Launching requested ML instancesProfilerReport-1635305525: InProgress
...
2021-10-27 03:33:00 Starting - Preparing the instances for training............
2021-10-27 03:34:59 Downloading - Downloading input data...
2021-10-27 03:35:34 Training - Downloading the training image...
2021-10-27 03:35:54 Training - Training image download completed. Training in progress.Docker entrypoint called with argument(s): train
Running default environment configuration script
[10/27/2021 03:36:00 INFO 139978396550976] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method'

In [10]:
from sagemaker.model import Model
from sagemaker.pipeline import PipelineModel
import boto3
from time import gmtime, strftime

timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

scikit_learn_inferencee_model = sklearn_preprocessor.create_model()
linear_learner_model = ll_estimator.create_model()

model_name = 'inference-pipeline-' + timestamp_prefix
endpoint_name = 'inference-pipeline-ep-' + timestamp_prefix
sm_model = PipelineModel(
    name=model_name, 
    role=role, 
    models=[
        scikit_learn_inferencee_model, 
        linear_learner_model])

sm_model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge', endpoint_name=endpoint_name)

-----------------!

In [11]:
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer


payload = '19,female,27.9,0,yes,southwest'
actual_charges = 16884.924
predictor = Predictor(
    endpoint_name=endpoint_name, sagemaker_session=sagemaker_session, serializer=CSVSerializer()
)

print(predictor.predict(payload))

b'{"predictions": [{"score": 25445.46484375}]}'


In [ ]:
sm_client = sagemaker_session.boto_session.client('sagemaker')
sm_client.delete_endpoint(EndpointName=endpoint_name)